In [1]:
import numpy as np
import plotly
import json
import os
import plotly.express as px
import numpy as np
import statistics as st
import pandas as pd



def write_to_csv(data, name):
    '''
    data in dict format, where keys form the column names
    '''
    df = pd.DataFrame(data)
    df.to_csv(name+'.csv', index=False)


########### trace processing ##########
def read_traces(log_path):
    '''
    read the trace files and extract variable names
    data = [ [event, timestamp], [], [],......,[] ]
    '''
    with open(log_path, 'r') as f:
        data = json.load(f)
    return data


def get_uniquevar(raw_trace):
    ''' 
    convert the v2.2 trace into list of unique variables
    raw_trace = data from read_traces, list( (var, ts),(var, ts),(var, ts),.... )
    return:
        unique_var = list(var1,var2,...) ## list of strings
    '''
    unique_var = []
    for rt in raw_trace:
        [var, timestamp] = rt
        # print([var, timestamp])
        if var not in unique_var:
            unique_var += [var]
            # print(rt)
    return unique_var


def generate_map(unique_events):
    '''
    unique_events -> list of all the variables in the code (unique, and in order of logging)
    return:
        event_map -> takes the variable name and gives corresponding event number
        event_remap -> takes event number and gives associated variable name
    '''
    event_map = dict()
    event_remap = dict()
    for i in range(len(unique_events)):
        event_remap[i+1] = unique_events[i]
        event_map[unique_events[i]] = i+1

    return(event_map, event_remap)


In [2]:
############ configuration ################
############################################

code = 'theft_protection'       ### application (code)
behaviour = 'faulty_data'            ### normal, faulty_data
thread_typ = 'single'           ### single, multi
version = 2.2                     ### format of data collection

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{code}/{thread_typ}_thread/version_{version}/{behaviour}'

### Get paths to the files

In [3]:
###### get file paths #######

all_files = os.listdir(log_path)
all_files.sort()
logs = []
traces = []
unknown = []
for i in all_files:
    if i.find('log') == 0:
        logs += [i]
    elif i.find('trace') == 0 and i.find('.txt') == -1:
        traces += [i]
    else:
        unknown += [i]

######### path to files
paths_log = [os.path.join(log_path, x) for x in logs]
paths_traces = [os.path.join(log_path, x) for x in traces]
paths_log.sort()

In [4]:
paths_traces

['../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace1-comm',
 '../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace2-bitflip',
 '../trace_data/theft_protection/single_thread/version_2.2/faulty_data/trace3-sensor']

In [5]:
### get variable list that is same for all log files ####
raw_trace = read_traces(paths_traces[0])
_var_list = get_uniquevar(raw_trace)
to_number, from_number = generate_map(_var_list)


### Process Traces

In [6]:
########## process the traces ###########

col_data = []
for (p,w) in zip(paths_traces, traces):
    trace = read_traces(p)
    num_trace = []
    time_stamp = []
    for (t, ts) in trace:
        nt = to_number[t]
        num_trace.extend([nt])
        time_stamp.extend([ts])
        # ### take limited samples
        # if ts > 250000:
        #     break
    col_data += [(w, time_stamp, num_trace, _var_list, p)]   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

In [ ]:
col_data[2]

### Generate plot trace data

In [7]:
all_df = []
for col in col_data:
    # print(col)
    plot_data = dict()
    plot_data['time'] = col[1]   ### x_data
    plot_data[col[0]] = col[2]   ### y_data (traces)

    ### convert the list to data frame and store it for plotting
    df = pd.DataFrame(plot_data)
    all_df += [df]

    #break


In [ ]:
all_df[0].columns[1]



### plot data in plotly

In [ ]:
# import plotly.graph_objects as go
# import pandas as pd


# # Create figure
# fig = go.Figure()

# ### add all the traces to the graph
# for df in all_df:
#     df_col = df.columns
#     fig.add_trace(
#                 go.Scatter( x=list(df[df_col[0]]), y=list(df[df_col[1]]))   ### equivalent to: x=list(df['time']), y=list(df['trace1'])
#                 )
    

# # Add range slider, title, yticks, axes labels
# fig.update_layout(
#     title_text="Event Trace Time",
#     xaxis=dict(
#         title="Number of events",
#         rangeslider=dict(visible=True),
#         type='linear'
#     ),
#     yaxis=dict(
#         title="Variables",
#         tickvals=[k for k in range(1,len(_var_list)+1)],
#         ticktext=_var_list
#     ),
#     autosize=True,
#     #width=500,
#     height=600,
    
# )


# # style all the traces
# fig.update_traces(
#     #hoverinfo="name+x+text",
#     line={"width": 0.5},
#     marker={"size": 8},
#     mode="lines+markers",
#     showlegend=True,
# )

# fig.show()


In [12]:
import plotly.graph_objects as go

import pandas as pd




### add all the traces to the graph
for df in all_df:

    # Create figure
    fig = go.Figure()

    df_col = df.columns
    fig.add_trace(
                go.Scatter(y=list(df[df_col[1]]), name=df_col[1])   ### equivalent to: y=list(df['trace1'])
                )

    # break
    ### generate x ticks with timestamp and index num  
    x_data = df[df_col[0]]
    x_ticks = [(i,x_data[i]) for i in range(0,len(x_data),10) ]

    # Add range slider, title, yticks, axes labels
    fig.update_layout(
        title_text="Event Trace without Time",
        xaxis=dict(
            title="Number of events",
            rangeslider=dict(visible=True),
            type='linear',
            tickvals=[k for k in range(0,len(x_data),10)],
            ticktext=x_ticks
        ),
        yaxis=dict(
            title="Variables",
            tickvals=[k for k in range(1,len(_var_list)+1)],
            ticktext=_var_list
        ),
        autosize=True,
        width=1200,
        height=600,
        
    )


    # style all the traces
    fig.update_traces(
        #hoverinfo="name+x+text",
        line={"width": 0.5},
        marker={"size": 8},
        mode="lines+markers",
        showlegend=True,
        
    )

    fig.show()


In [ ]:
x_ticks

### Generate excel sheet

In [ ]:
######### write the data to excel sheet ###########

import xlsxwriter

workbook = xlsxwriter.Workbook(f'{thread_typ}_version{version}_eventtrace_{behaviour}.xlsx')

### add worksheet for mapper
worksheet = workbook.add_worksheet('mapper')

### convert dict to list and write to xls

worksheet.write(0, 0, 'event num')
worksheet.write(0, 1, 'event name')

row = 1
col = 0
keym = list(from_number.keys())
for k in keym:
    val = from_number[k]
    worksheet.write(row, col, k)
    worksheet.write(row, col+1, val)
    row+=1

### add worksheet for traces
worksheet = workbook.add_worksheet('traces')
row = 0
col = 0
for (name, time, trace, labels) in col_data:
    worksheet.write(row, col, name)
    row=1
    worksheet.write(row, col, 'events')
    row=2
    for t in trace:
        #worksheet.write(row, col, from_number[t])
        worksheet.write(row, col, t)
        row+=1
    row=1
    col+=1
    worksheet.write(row, col, 'timestamp')
    row=2
    for t in time:
        worksheet.write(row, col, t)
        row+=1

    col+=2
    row=0

workbook.close()

### Variable wise plots

In [ ]:
paths_traces

In [8]:
from collections import defaultdict 

tl_data = []     ### [log1, log,2, ... ] --> [{var1:[], var2:[], ....}, {}  ]

for (p,w) in zip(paths_traces, traces):
    #print(p,w)
    trace = read_traces(p)
    #print(trace)
    var_timelist = defaultdict(list)
    # for var_name in _var_list:
    #     print(var_name)
    for ind, (t, ts) in enumerate(trace):
        #print(t,ts)
        var_timelist[t] += [[ts, ind]]     ### format: {var1:[[ts, ind], ...], var2:[[ts, ind], ...]}

    tl_data += [var_timelist]    
    #break


In [ ]:
tl_data

In [9]:
############ prepare data to plot #############
###############################################

to_plot = []   ### in format -> [var_name, ( [[<exe inters of var in log1>], [timestamps ]  )]

#### use _var_list for consistency

### collect data for each variable from each log file
for v in _var_list:
    xy_data = [] ### execution intervals
    log_names = []
    for i, (p,w, data) in enumerate(zip(paths_log, logs, tl_data)):
        # print('i:',i)

        ##############################
        if i==0:   ### take data of single log for plotting
        #################################

            try:
                #_, data = read_logs(p)   ### data of each log file
                time_list = data[v]   ### get the timestamps
                #print(data)
                exe_time = []
                timestamp = []
                indices = []

                # print(len(data))
                # print(time_list)
        
                for _, (td1,td2) in enumerate(zip(time_list[0:-1], time_list[1:])):
                    t2 = td2[0]
                    t1 = td1[0]
                    ind = td2[1]
                    tdiff = t2-t1
                    #print(tdiff)
                    exe_time+=[tdiff]
                    timestamp+=[t2] 
                    indices+=[ind]  ### index of the event in trace files | to maps events between full trace plots and variable plots
                    
                    # print(ind,t2)
                    # ##### testing limited samples  
                    # if t2 > 250000:
                    #     break
                    # ###### testing limited samples

                print('length of exe_time:', len(exe_time))
                assert(len(exe_time)==len(timestamp))
                log_names += [w]
                xy_data += [(exe_time,timestamp, indices)]
            except Exception as e:
                print(f'{w}:{v}', e)    ### will execute if any variable has only one execution time
        
    assert(len(log_names)==len(xy_data))
    to_plot += [(p.replace(w,v),log_names,xy_data)]  ### [name of the file to write plots(variable name), labels for legend (log names), execution intervals for respective variables(y_data), timestamps(x_data, indexs)]



length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 2000
length of exe_time: 1999
length of exe_time: 1999
length of exe_time: 1996
length of exe_time: 424
length of exe_time: 424
length of exe_time: 424
length of exe_time: 305
length of exe_time: 305
length of exe_time: 305


In [ ]:
for x in to_plot[6:]:
    print(len(x))
    print(x[2][0][2])
    break

In [11]:
import plotly.graph_objects as go
import pandas as pd


### name represents the name of respective variable with which file will be saved
for (name, log_names, xy_data) in to_plot:
    ### path to save the plots
    to_write_name = name.replace('trace_data', 'exe plots')
    file_name = os.path.basename(to_write_name)
    file_name = f'{thread_typ}_version{version}_{behaviour}_{file_name}'
    dir_name = os.path.dirname(to_write_name)
    to_write_name = os.path.join(dir_name, file_name)
    #print(to_write_name)
    isPath = os.path.exists(os.path.dirname(to_write_name)) ### check if the path exists
    ### create the folder if it does not exist
    if not isPath:
        os.makedirs(os.path.dirname(to_write_name))

    
    ########## make data frame to be able to plot ################
    df = dict()
    _y_all = [] ### to adjust y-ticks
    legend_lab = [] ### collect names of the plots only
    line_style = ['solid', 'dashed', 'dashdot', 'dotted']
    markers = ['.','o','*','+','^','x','d','h',',','H','D']

    # Create figure
    fig = go.Figure()
    # print(xy_data)
    for (num, (l,xy)) in enumerate(zip(log_names, xy_data)):
        x = xy[1]
        #x = [i-x[0] for i in x]   ### get timestamps relative to first timestamp
        y = xy[0]
        ind = xy[2]
        # print(len(x),len(y),len(ind))

        x_ticks = [(ind[i],x[i]) for i in range(0,len(x),5) ]
        print(x_ticks)

        ### ignore all the variables that are only executed once
        if xy[1]!= []:
            #print(x,y)
            df[l]=xy
            _y_all.extend(y)
            legend_lab.append(l)
            
            # plt.plot(x, y, ls=line_style[num%4], marker=markers[num%11])
            fig.add_trace(
                go.Scatter(y=y, name=l))
            
            # Add range slider, title, yticks, axes labels
            fig.update_layout(
                title_text=f"Execution Interval for '{os.path.basename(name)}'",
                xaxis=dict(
                    title="Time (in ms)",
                    rangeslider=dict(visible=True),
                    type='linear',
                    tickvals=[k for k in range(0,len(x),5)],
                    ticktext=x_ticks
                    ),
                yaxis=dict(
                    title="Execution interval (ms)",
                    # tickvals=[k for k in range(1,len(_var_list)+1)],
                    # ticktext=_var_list
                    ),
                autosize=True,
                #width=500,
                #height=600,
                )
        
    if _y_all != []:

        # style all the traces
        fig.update_traces(
            #hoverinfo="name+x+text",
            line={"width": 0.5},
            marker={"size": 8},
            mode="lines+markers",
            showlegend=True,
            )

        fig.show()

        # break

[]
[]
[]
[]
[]
[]
[(10, 3013), (36, 8290), (62, 13569), (88, 18847), (114, 24126), (140, 29406), (166, 34685), (192, 39966), (218, 45245), (244, 50526), (270, 55809), (296, 61089), (322, 66368), (348, 71648), (374, 76928), (403, 82223), (429, 87504), (455, 92784), (484, 98077), (510, 103359), (536, 108641), (562, 113920), (588, 119201), (614, 124483), (640, 129764), (666, 135043), (692, 140323), (718, 145604), (744, 150885), (770, 156168), (796, 161450), (822, 166732), (851, 172025), (877, 177307), (903, 182590), (929, 187870), (958, 193165), (984, 198446), (1010, 203729), (1036, 209010), (1062, 214291), (1088, 219573), (1114, 224852), (1140, 230134), (1166, 235414), (1192, 240696), (1218, 245977), (1244, 251257), (1270, 256540), (1296, 261822), (1325, 267117), (1351, 272398), (1377, 277679), (1403, 282963), (1432, 288257), (1458, 293540), (1484, 298823), (1510, 304107), (1536, 309391), (1562, 314674), (1588, 319955), (1614, 325235), (1640, 330523), (1666, 335803), (1692, 341085), (171

[(11, 3038), (37, 8316), (63, 13594), (89, 18872), (115, 24151), (141, 29431), (167, 34710), (193, 39991), (219, 45270), (245, 50551), (271, 55834), (297, 61114), (323, 66393), (349, 71673), (375, 76953), (404, 82248), (430, 87529), (456, 92809), (485, 98103), (511, 103384), (537, 108666), (563, 113945), (589, 119226), (615, 124508), (641, 129789), (667, 135068), (693, 140348), (719, 145629), (745, 150910), (771, 156193), (797, 161475), (823, 166757), (852, 172050), (878, 177333), (904, 182615), (930, 187895), (959, 193190), (985, 198472), (1011, 203754), (1037, 209036), (1063, 214316), (1089, 219598), (1115, 224877), (1141, 230159), (1167, 235439), (1193, 240721), (1219, 246002), (1245, 251282), (1271, 256565), (1297, 261847), (1326, 267142), (1352, 272423), (1378, 277704), (1404, 282988), (1433, 288283), (1459, 293566), (1485, 298848), (1511, 304132), (1537, 309416), (1563, 314699), (1589, 319980), (1615, 325260), (1641, 330548), (1667, 335828), (1693, 341110), (1719, 346392), (1745,

[(12, 4054), (38, 9332), (64, 14611), (90, 19888), (116, 25167), (142, 30448), (168, 35727), (194, 41008), (220, 46287), (246, 51568), (272, 56851), (298, 62130), (324, 67410), (350, 72689), (376, 77970), (405, 83266), (431, 88546), (457, 93826), (486, 99119), (512, 104401), (538, 109683), (564, 114962), (590, 120244), (616, 125524), (642, 130805), (668, 136084), (694, 141365), (720, 146646), (746, 151928), (772, 157210), (798, 162491), (824, 167773), (853, 173067), (879, 178349), (905, 183632), (931, 188912), (960, 194207), (986, 199488), (1012, 204772), (1038, 210053), (1064, 215332), (1090, 220615), (1116, 225893), (1142, 231175), (1168, 236456), (1194, 241738), (1220, 247018), (1246, 252299), (1272, 257582), (1298, 262864), (1327, 268159), (1353, 273440), (1379, 278721), (1405, 284005), (1434, 289299), (1460, 294582), (1486, 299865), (1512, 305149), (1538, 310433), (1564, 315716), (1590, 320997), (1616, 326277), (1642, 331564), (1668, 336845), (1694, 342127), (1720, 347409), (1746,

[(13, 4058), (39, 9337), (65, 14615), (91, 19893), (117, 25172), (143, 30453), (169, 35732), (195, 41012), (221, 46291), (247, 51572), (273, 56856), (299, 62134), (325, 67415), (351, 72694), (377, 77974), (406, 83271), (432, 88550), (458, 93831), (487, 99124), (513, 104406), (539, 109687), (565, 114967), (591, 120248), (617, 125528), (643, 130809), (669, 136088), (695, 141370), (721, 146650), (747, 151932), (773, 157215), (799, 162495), (825, 167778), (854, 173071), (880, 178355), (906, 183636), (932, 188917), (961, 194211), (987, 199493), (1013, 204776), (1039, 210057), (1065, 215337), (1091, 220619), (1117, 225897), (1143, 231179), (1169, 236460), (1195, 241742), (1221, 247023), (1247, 252304), (1273, 257586), (1299, 262869), (1328, 268163), (1354, 273444), (1380, 278726), (1406, 284010), (1435, 289304), (1461, 294587), (1487, 299870), (1513, 305154), (1539, 310437), (1565, 315720), (1591, 321001), (1617, 326281), (1643, 331569), (1669, 336849), (1695, 342131), (1721, 347414), (1747,

[(52, 12492), (178, 37837), (300, 62138), (426, 87490), (552, 112843), (674, 137143), (800, 162499), (926, 187856), (1048, 212161), (1174, 237514), (1300, 262872), (1422, 287180), (1548, 312543), (1674, 337903), (1791, 362046), (1917, 387412), (2043, 412778), (2165, 437093), (2291, 462456), (2417, 487819), (2539, 512136), (2665, 537503), (2786, 563163), (2908, 587482), (3034, 612860), (3157, 638227), (3279, 662558), (3393, 687894), (3504, 713209), (3617, 737504), (3740, 762884), (3863, 788273), (3970, 812539), (4081, 837859), (4204, 863294), (4326, 887703), (4452, 913169), (4574, 937573), (4700, 963030), (4819, 987412), (4930, 1012732), (5041, 1038052), (5152, 1063380), (5259, 1087725), (5376, 1113228), (5498, 1137821), (5620, 1162420), (5746, 1188065), (5868, 1212656), (5994, 1238303), (6116, 1262896), (6238, 1287497), (6361, 1313096), (6463, 1337717), (6574, 1363120), (6681, 1387469), (6795, 1412937), (6917, 1437619), (7043, 1463356), (7165, 1488038), (7287, 1512722), (7409, 1537392)

[(53, 12497), (179, 37842), (301, 62143), (427, 87495), (553, 112848), (675, 137148), (801, 162504), (927, 187861), (1049, 212166), (1175, 237519), (1301, 262877), (1423, 287185), (1549, 312548), (1675, 337908), (1792, 362052), (1918, 387417), (2044, 412783), (2166, 437098), (2292, 462461), (2418, 487824), (2540, 512141), (2666, 537508), (2787, 563168), (2909, 587487), (3035, 612865), (3158, 638232), (3280, 662563), (3394, 687899), (3505, 713214), (3618, 737509), (3741, 762889), (3864, 788278), (3971, 812544), (4082, 837864), (4205, 863300), (4327, 887708), (4453, 913174), (4575, 937578), (4701, 963035), (4820, 987418), (4931, 1012737), (5042, 1038057), (5153, 1063385), (5260, 1087731), (5377, 1113233), (5499, 1137826), (5621, 1162425), (5747, 1188071), (5869, 1212661), (5995, 1238309), (6117, 1262902), (6239, 1287503), (6362, 1313102), (6464, 1337723), (6575, 1363125), (6682, 1387474), (6796, 1412943), (6918, 1437624), (7044, 1463361), (7166, 1488044), (7288, 1512727), (7410, 1537398)

[(54, 12502), (180, 37847), (302, 62148), (428, 87500), (554, 112853), (676, 137153), (802, 162508), (928, 187866), (1050, 212171), (1176, 237524), (1302, 262882), (1424, 287189), (1550, 312553), (1676, 337912), (1793, 362056), (1919, 387422), (2045, 412788), (2167, 437103), (2293, 462466), (2419, 487828), (2541, 512145), (2667, 537512), (2788, 563173), (2910, 587492), (3036, 612870), (3159, 638237), (3281, 662568), (3395, 687904), (3506, 713219), (3619, 737514), (3742, 762894), (3865, 788283), (3972, 812549), (4083, 837869), (4206, 863304), (4328, 887713), (4454, 913179), (4576, 937583), (4702, 963040), (4821, 987422), (4932, 1012742), (5043, 1038062), (5154, 1063390), (5261, 1087736), (5378, 1113237), (5500, 1137831), (5622, 1162429), (5748, 1188076), (5870, 1212666), (5996, 1238313), (6118, 1262906), (6240, 1287508), (6363, 1313106), (6465, 1337728), (6576, 1363130), (6683, 1387478), (6797, 1412947), (6919, 1437629), (7045, 1463366), (7167, 1488048), (7289, 1512732), (7411, 1537402)

[(59, 13556), (185, 38902), (307, 63202), (433, 88554), (559, 113907), (681, 138208), (807, 163563), (933, 188920), (1055, 213227), (1181, 238580), (1307, 263937), (1429, 288245), (1555, 313610), (1681, 338968), (1798, 363112), (1924, 388478), (2050, 413843), (2172, 438157), (2298, 463521), (2424, 488885), (2546, 513201), (2672, 538567), (2793, 564230), (2915, 588547), (3041, 613927), (3186, 643526), (3576, 729043), (3721, 758661), (4111, 844201), (4233, 868609), (4359, 894075), (4481, 918484), (4607, 943943), (4733, 969400), (5383, 1114297), (5505, 1138891), (5627, 1163489), (5753, 1189135), (5875, 1213726), (6001, 1239373), (6123, 1263966), (6245, 1288567), (6802, 1414007), (6924, 1438689), (7050, 1464426), (7172, 1489108), (7294, 1513792), (7416, 1538473), (7542, 1564201), (7928, 1649257), (8050, 1673998), (8172, 1698738), (8535, 1778554), (8661, 1804403), (8783, 1829184), (9062, 1889995), (9180, 1913767), (9501, 1984130), (9623, 2008997), (9745, 2033847), (10085, 2108497)]


[(60, 13561), (186, 38906), (308, 63207), (434, 88559), (560, 113912), (682, 138213), (808, 163568), (934, 188925), (1056, 213232), (1182, 238584), (1308, 263942), (1430, 288249), (1556, 313614), (1682, 338972), (1799, 363116), (1925, 388482), (2051, 413847), (2173, 438162), (2299, 463526), (2425, 488890), (2547, 513205), (2673, 538572), (2794, 564234), (2916, 588551), (3042, 613931), (3187, 643531), (3577, 729048), (3722, 758665), (4112, 844206), (4234, 868613), (4360, 894080), (4482, 918488), (4608, 943948), (4734, 969405), (5384, 1114301), (5506, 1138895), (5628, 1163494), (5754, 1189140), (5876, 1213730), (6002, 1239378), (6124, 1263971), (6246, 1288572), (6803, 1414012), (6925, 1438694), (7051, 1464431), (7173, 1489113), (7295, 1513797), (7417, 1538478), (7543, 1564205), (7929, 1649261), (8051, 1674003), (8173, 1698742), (8536, 1778559), (8662, 1804408), (8784, 1829188), (9063, 1889999), (9181, 1913772), (9502, 1984135), (9624, 2009001), (9746, 2033852), (10086, 2108502)]


[(61, 13565), (187, 38910), (309, 63211), (435, 88563), (561, 113916), (683, 138216), (809, 163572), (935, 188929), (1057, 213236), (1183, 238588), (1309, 263946), (1431, 288253), (1557, 313618), (1683, 338976), (1800, 363120), (1926, 388486), (2052, 413851), (2174, 438166), (2300, 463530), (2426, 488894), (2548, 513209), (2674, 538576), (2795, 564238), (2917, 588555), (3043, 613935), (3188, 643535), (3578, 729052), (3723, 758669), (4113, 844210), (4235, 868618), (4361, 894084), (4483, 918492), (4609, 943952), (4735, 969409), (5385, 1114305), (5507, 1138899), (5629, 1163498), (5755, 1189144), (5877, 1213734), (6003, 1239382), (6125, 1263983), (6247, 1288576), (6804, 1414015), (6926, 1438698), (7052, 1464434), (7174, 1489116), (7296, 1513801), (7418, 1538482), (7544, 1564210), (7930, 1649266), (8052, 1674007), (8174, 1698746), (8537, 1778563), (8663, 1804412), (8785, 1829193), (9064, 1890003), (9182, 1913776), (9503, 1984139), (9625, 2009006), (9747, 2033856), (10087, 2108506)]
